In [103]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import mahalanobis as maha

## Data Cleaning

- Load Data
- Calculate yield curve slope and Y/Y changes in relevant columns


#### Data Definitions
- TNX: US 10y Treasury
- US_Corp: ML US Corporate Bond Total Return Index
- LIBOR: 3m LIBOR Rate
- BAA: Moody's long-term corporate bond yields index
- UNRATE: US seasonally-adjusted unemployment rate
- SPY: S\&P 500 Index
- IRX: US 3m treasury rate
- RGDP: US seasonally-adjusted Real GDP


In [204]:
#Missing Commodity Index Data
data = pd.read_csv('data/data.csv', index_col=0)
data.index = pd.to_datetime(data.index)

In [205]:
#Linear Interpolation Forward fill - for GDP data which is quarterly
#Could potentially want to only use quarterly data - set lin_interp = False
lin_interp = True

if lin_interp:
    data['RGDP'] = data['RGDP'].interpolate()
else:
    data = data.dropna(subset = ['RGDP'])

In [206]:
#Yield Curve Slope: 10y yields - 3m yields
data['YC_Slope'] =  pd.eval('data.TNX - data.IRX')
#Credit Spread: long-term BAA (corp bonds) - 10y treasury rate
data['Cred_Spread'] = pd.eval('data.BAA - data.TNX')

In [207]:
#Fill in Y/Y changes
YY_cols = ['CPI', 'RGDP']
data[[x + '_Growth' for x in YY_cols]] = data[YY_cols]/data[YY_cols].shift(12 if lin_interp else 4) - 1

#Drop null rows
data = data.dropna()

#Subtract mean
# data = data - data.mean()

In [208]:
data.median()

TNX                4.125500
US_Corp         1541.163409
LIBOR              2.398775
CPI              205.650000
BAA                6.585000
UNRATE             5.400000
SPY              125.500000
IRX                1.868500
RGDP           15180.037833
YC_Slope           1.746500
Cred_Spread        2.338500
CPI_Growth         0.021842
RGDP_Growth        0.025657
dtype: float64

In [209]:
data.std()

TNX               1.652370
US_Corp         738.421893
LIBOR             2.200002
CPI              32.089959
BAA               1.404468
UNRATE            1.631899
SPY              62.351690
IRX               2.059814
RGDP           2547.039609
YC_Slope          1.080416
Cred_Spread       0.778178
CPI_Growth        0.005264
RGDP_Growth       0.016129
dtype: float64

In [246]:
#Quick function to get num_sd standard deviations away from the median.
#Up pos determines if up is good or bad. Idea being that good scenario is at index 1, bad scenario at index 0.
def get_range(data, variable, num_sd, up_pos = True):
    i = 1 if up_pos else -1
    
    v = data[variable]
    v = v[(v < v.quantile(.75) ) & (v > v.quantile(.25))]
    m = v.mean()
    s = v.std()
    
    return tuple((m - i * num_sd * s, m + i * num_sd * s))

get_range(data, 'UNRATE', 1, False)

(5.856824259398638, 4.9444829301438515)

## Defining Scenarios

- Define a scenario with a boolean string. Make sure to use spaces between operators (for calculation of mahalanobis distances.
- Find empirical mean and covariance matrix of the factors in this scenario.
- Encode scenario as a vector, then find mahalanobis distance using scenario vector and empirical mean, covariance.
- We then convert scenario Mahalanobis distance into likelihood measure:
$$ e^{\frac{-d}{2}}$$
- Rescale probabilities to sum to 1

In [215]:
def get_scenario_vars(scenario):
    #Get the variables in the scenario. Sort alphabetically for consistent replication
    return sorted(list(set([v.split(' ')[0] for v in scenario.split(' & ')])))

def get_scenario_vector(scenario):
    #Get a vector from a scenario.
    vector = {}
    for v in scenario.split(' & '):
        x = v.split(' ')
        cn = x[0]
        val = x[2]
        vector[cn] = float(val)
    vector = pd.DataFrame(vector, index = [0])
    colnames = get_scenario_vars(scenario)
    return vector[colnames].values

In [252]:
#Build the scenarios: Weak is >= 1 sd below median, strong is >= 1 sd above median, normal is in range [m - sd, m + sd].

scen_names = ['Weak', 'Strong', 'Normal']
# relevant_vars = ['RGDP_Growth', 'UNRATE', 'CPI_Growth', 'YC_Slope', 'Cred_Spread', 'TNX']
relevant_vars = ['RGDP_Growth', 'UNRATE', 'CPI_Growth']
# relevant_vars = ['RGDP_Growth', 'UNRATE']

scenarios = {sn: '' for sn in scen_names}

up_pos = {v:True for v in relevant_vars}
up_pos['UNRATE'] = False
up_pos['YC_Slope'] = False
up_pos['Cred_Spread'] = False

first_run = True

for var in relevant_vars:
    if first_run:
        first_run = False
    else:
        for v in scen_names:
            scenarios[v] += ' & '
    
    low_bound, u_bound = get_range(data, var, 1, up_pos[var])
    
    l_sign = '<=' if up_pos[var] else '>='
    u_sign = '>=' if up_pos[var] else '<='
    
    scenarios['Weak'] += ('{} {} {}'.format(var, l_sign, low_bound))
    scenarios['Normal'] += ('{} {} {} & {} {} {}'.format(var, u_sign, low_bound, var, l_sign, u_bound))
    scenarios['Strong'] += ('{} {} {}'.format(var, u_sign, u_bound))
    
    print(var, low_bound, u_bound)
    
scenarios

RGDP_Growth 0.02153049417710918 0.03118893941453791
UNRATE 5.856824259398638 4.9444829301438515
CPI_Growth 0.020055987437858774 0.023444747654114598


{'Weak': 'RGDP_Growth <= 0.02153049417710918 & UNRATE >= 5.856824259398638 & CPI_Growth <= 0.020055987437858774',
 'Strong': 'RGDP_Growth >= 0.03118893941453791 & UNRATE <= 4.9444829301438515 & CPI_Growth >= 0.023444747654114598',
 'Normal': 'RGDP_Growth >= 0.02153049417710918 & RGDP_Growth <= 0.03118893941453791 & UNRATE <= 5.856824259398638 & UNRATE >= 4.9444829301438515 & CPI_Growth >= 0.020055987437858774 & CPI_Growth <= 0.023444747654114598'}

In [253]:
likelihoods = {}
l_sum = 0.0

for scenario_name, scenario in scenarios.items():
    #Get relevant variables for scenario defined above
    scenario_vars = get_scenario_vars(scenario)

    #Get the empirical mean & correlation matrix of scenario vars
    v = data[scenario_vars].mean().values
    scen_corr = data[scenario_vars].corr()

    #Encode the scenario as a vector - take empirical averages after conditioning on scenario
    u = data.query(scenario)[scenario_vars].mean().values
    print(u)
    #u = get_scenario_vector(scenario)

    #Mahalanobis distance, converted to likelihood
    l = np.exp(-maha(u, v, scen_corr)/2)
    likelihoods[scenario_name] = l
    l_sum += l

probs = {sn: l/l_sum for sn, l in likelihoods.items()}
print(likelihoods)
print(probs)
# print(mahala)

[1.65303715e-02 4.93656375e-03 8.28837209e+00]
[0.0247467  0.04394743 4.3       ]
[0.02199571 0.02214366 5.8       ]
{'Weak': 0.2816571428182477, 'Strong': 0.4785803154302323, 'Normal': 0.9819058598498099}
{'Weak': 0.1616727739286699, 'Strong': 0.27470777545021197, 'Normal': 0.5636194506211181}


In [254]:
# scenario = 'RGDP_Growth = 0.01 & UNRATE = 6'

### Disregard below - not right.

In [255]:
#Define scenario, use spaces between operators for ease of parsing.
scenario = 'RGDP_Growth = 0.01 & UNRATE = 6'

#Get relevant variables from the scenario defined above.
scenario_vars = [v.split(' ')[0] for v in scenario.split(' & ')]

#Filter data on the above scenario - not sure if useful or not...
scen_data = data.query(scenario)[scenario_vars]

#Get the empirical mean & correlation matrix of scenario vars
v = data[scenario_vars].mean().values
scen_corr = data[scenario_vars].corr()

#Calculate mahalanobis distance, transform to likelihood measure
data['L_1'] = data[scenario_vars].apply(lambda x: np.exp(-maha(x ,v, scen_corr)/2), raw = True, axis = 1)

#Rescale for likelihood
data['Prob_1'] = data['L_1'].dropna()/data['L_1'].sum()
# data['Prob_1'] = data['Prob_1'].fillna(0)
data.Prob_1

Date
1993-01-01    0.007402
1993-04-01    0.008183
1993-07-01    0.009043
1993-10-01    0.009513
1994-01-01    0.010511
1994-04-01    0.011619
1994-07-01    0.013492
1994-10-01    0.015674
1995-01-01    0.014360
1995-04-01    0.015669
1995-07-01    0.015091
1995-10-01    0.013657
1996-01-01    0.014356
1996-04-01    0.014343
1996-07-01    0.013654
1996-10-01    0.011751
1997-01-01    0.012357
1997-04-01    0.011172
1997-07-01    0.010112
1997-10-01    0.009153
1998-01-01    0.008706
1998-04-01    0.007493
1998-07-01    0.008279
1998-10-01    0.008277
1999-01-01    0.007493
1999-04-01    0.007494
1999-07-01    0.007491
1999-10-01    0.006776
2000-01-01    0.006453
2000-04-01    0.005832
                ...   
2012-04-01    0.004721
2012-07-01    0.004720
2012-10-01    0.005764
2013-01-01    0.005219
2013-04-01    0.006371
2013-07-01    0.007406
2013-10-01    0.007785
2014-01-01    0.010500
2014-04-01    0.012841
2014-07-01    0.012840
2014-10-01    0.015094
2015-01-01    0.015092
2015-0

In [23]:
scenario_1 = 'RGDP_Growth = 0.01 & UNRATE = 6'
scenario_2 = 'RGDP_Growth '

# l_1 = np.exp(-maha(np.array([float(v.split(' ')[2]) for v in scenario_1.split(' & ')]), v, scen_corr))

In [25]:
get_scenario_vars(scenario_1)

['RGDP_Growth', 'UNRATE']

In [ ]:
get_s